In [43]:
import os
import sys
import torchvision
from transformers.tokenization_bert import BertTokenizer

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/work'

In [44]:
sys.path.append(w_dir+'/fgc_support_retri')

In [45]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
from transformers.tokenization_bert import BertTokenizer
import config
from fgc_preprocess import FgcSerDataset, BertIdx
from torch.utils.data import DataLoader
from sup_model import BertSupSentClassification
from transformers import BertModel
from tqdm import tqdm_notebook as tqdm
import torch
import torch.nn as nn
import torchvision
from transformers.tokenization_bert import BertTokenizer
import config
from fgc_preprocess import FgcSerDataset, BertIdx
from torch.utils.data import DataLoader
from sup_model import BertSupSentClassification
from transformers import BertModel
from transformers.optimization import AdamW, WarmupLinearSchedule
from tqdm import tqdm
import ujson
import json
import config
import numpy as np
import torch
from tqdm import tqdm_notebook as tqdm
from torch.utils.data import Dataset
from stanfordcorenlp import StanfordCoreNLP

In [46]:
nlp = StanfordCoreNLP('http://140.109.19.191', port=9000, lang='zh')
props = {'annotators': 'ssplit', 'ssplit.boundaryTokenRegex': '[。]|[!?！？]+',
         'outputFormat': 'json', 'pipelineLanguage': 'zh', 'timeout': '5000000'}

def sentence_split(dtext):
    anno = json.loads(nlp.annotate(dtext))

    out_sents = []
    for s in anno['sentences']:
        s_start = s['tokens'][0]['characterOffsetBegin']
        s_end = s['tokens'][-1]['characterOffsetEnd']
        s_string = dtext[s_start: s_end]

        out_sents.append(s_string)

    return out_sents

I1126 04:17:15.763602 139662184884032 corenlp.py:42] Using an existing server http://140.109.19.191:9000
I1126 04:17:16.767912 139662184884032 corenlp.py:118] The server is available.


In [47]:
def convert2ids(tokenizer, sentences, question):
    input_ids_all = torch.zeros((len(sentences), 512), dtype=torch.long)
    token_type_ids_all = torch.zeros((len(sentences), 512), dtype=torch.long)
    attention_mask_all = torch.zeros((len(sentences), 512), dtype=torch.long)
    
    for s_i, sentence in enumerate(sentences):
        input_ids = torch.zeros(512, dtype=torch.long)
        token_type_ids = torch.zeros(512, dtype=torch.long)
        attention_mask = torch.zeros(512, dtype=torch.long)

        tokenized_q = ['[CLS]'] + tokenizer.tokenize(question) + ['[SEP]']
        tokenized_all = tokenized_q + tokenizer.tokenize(sentence)

        if len(tokenized_all) > 511:
            print("tokenized all > 511")
            tokenized_all = tokenized_all[:512]
        tokenized_all += ['[SEP]']
        ids_all = tokenizer.convert_tokens_to_ids(tokenized_all)

        input_ids[:len(ids_all)] = torch.tensor(ids_all)
        token_type_ids[len(tokenized_q):len(tokenized_all)] = 1
        attention_mask[:len(tokenized_all)] = 1
        
        input_ids_all[s_i] = input_ids
        token_type_ids_all[s_i] = token_type_ids
        attention_mask_all[s_i] = attention_mask

    return input_ids_all, token_type_ids_all, attention_mask_all

In [48]:
class SER_extract:
    def __init__(self):
        bert_model_name = 'bert-base-chinese'
        bert_encoder = BertModel.from_pretrained(bert_model_name)
        model = BertSupSentClassification(bert_encoder)
        model.load_state_dict(torch.load('trainedmodel.m'))
        model.eval()
        
        self.tokenizer = tokenizer = BertTokenizer.from_pretrained(bert_model_name)
        self.model = model
        
    def predict(self, context_sents, question):
        input_ids, token_type_ids, attention_mask = convert2ids(self.tokenizer, context_sents, question)
        
        with torch.no_grad():
            logits = self.model(input_ids, token_type_ids=token_type_ids,
						 attention_mask=attention_mask, mode=BertSupSentClassification.ForwardMode.EVAL)
        return logits

In [49]:
ser_extracter = SER_extract()

I1126 04:17:28.513021 139662184884032 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.0c16faba8be66db3f02805c912e4cf94d3c9cffc1f12fa1a39906f9270f76d33
I1126 04:17:28.515698 139662184884032 configuration_utils.py:168] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "fi

In [84]:
from fgc_preprocess import prepro_all

items = prepro_all(config.FGC_TEST)

preprocessing /work/fgc_support_retri/data/FGC/1.2/FGC_release_A_test(cn).json ......


Expecting value: line 1 column 1 (char 0)
D039
data size = 165


In [88]:
for item in tqdm(items):
    del item['QUESTIONS']

In [91]:
for item in tqdm(items):
    item['SENTS'] = [s for s, t, e in item['SENTS']]

In [102]:
items[10]

{'DTEXT': '阿拉伯之春，又称「阿拉伯的觉醒」、「阿拉伯起义」，是指自2010年年底在北非和西亚的阿拉伯国家发生的一系列以「民主」和「经济」等为主题的社会运动，这些运动多采取公开示威游行和网络串连的方式，其影响之深、范围之广使全世界十分关注，从2011年初开始，其带来的影响与后续效应至今尚未完全结束。\n这场运动以北非国家突尼西亚的班·阿里政权被民众抗议推翻为肇始，形成一场规模空前的民众反政府运动，如潮水般席卷整个阿拉伯世界，稳坐江山数十年之久的政治强人和独裁者如多米诺骨牌般接二连三地倒下。革命运动浪潮随后波及埃及、利比亚、叶门、叙利亚、阿尔及利亚、苏丹、巴林、沙乌地阿拉伯、阿曼、伊拉克、茅利塔尼亚、约旦、摩洛哥、科威特、黎巴嫩等国。其他一些阿拉伯国家乃至部分非阿拉伯国家也都受到不同程度的影响，发生了一些抗议示威活动，但大都因政府的让步或示威规模不大而逐渐平息。\n在这次大规模示威中，由于脸书、推特、YouTube等社交媒体发挥了重要作用，一些国家（如埃及和利比亚）对该国部分地区或全国网络进行了封锁，导致该国网络与世界隔绝。\n',
 'DID': 'D007',
 'SENTS': ['阿拉伯之春，又称「阿拉伯的觉醒」、「阿拉伯起义」，是指自2010年年底在北非和西亚的阿拉伯国家发生的一系列以「民主」和「经济」等为主题的社会运动，这些运动多采取公开示威游行和网络串连的方式，其影响之深、范围之广使全世界十分关注，从2011年初开始，其带来的影响与后续效应至今尚未完全结束。',
  '这场运动以北非国家突尼西亚的班·阿里政权被民众抗议推翻为肇始，形成一场规模空前的民众反政府运动，如潮水般席卷整个阿拉伯世界，稳坐江山数十年之久的政治强人和独裁者如多米诺骨牌般接二连三地倒下。',
  '革命运动浪潮随后波及埃及、利比亚、叶门、叙利亚、阿尔及利亚、苏丹、巴林、沙乌地阿拉伯、阿曼、伊拉克、茅利塔尼亚、约旦、摩洛哥、科威特、黎巴嫩等国。',
  '其他一些阿拉伯国家乃至部分非阿拉伯国家也都受到不同程度的影响，发生了一些抗议示威活动，但大都因政府的让步或示威规模不大而逐渐平息。',
  '在这次大规模示威中，由于脸书、推特、YouTube等社交媒体发挥了重要作用，一些国家（如埃及和利比亚）对该国部分地区或全国网络进行了封锁，导致该国网络与世界隔绝。'],
 'QTYPE'

In [101]:
tp = 0
gol_t = 0
pre_t = 0

for item in tqdm(items):
    logits = ser_extracter.predict(item['SENTS'], item['QTEXT'])
    prediction = torch.sigmoid(logits) > 0.5
    prediction = prediction.squeeze(1).numpy()
    gold = np.array(item['SUP_EVIDENCE'])

    gol_t += np.count_nonzero(gold == 1)
    pre_t +=  np.count_nonzero(prediction == 1)
    
    if len(gold) != len(prediction):
        print(gold)
        print(prediction)
        print(len(item['SENTS']))
        print(item['SENTS'])
        continue

    for i, gs in enumerate(gold):
        if gs == prediction[i] == 1:
            tp += 1
    item['prediction'] = prediction

print('precision = {}'.format(tp / pre_t))
print('recall = {}'.format(tp / gol_t))

KeyboardInterrupt: 

In [94]:
precision = tp / pre_t
recall = tp / gol_t

f1 = 2*precision*recall / (precision+recall)

print("precision = {}".format(precision))
print("recall = {}".format(recall))
print("f1 = {}".format(f1))

precision = 0.2898089171974522
recall = 0.3855932203389831
f1 = 0.33090909090909093


In [95]:
d_num = 0
q_num = 0
s_num = 0
evidence_num = 0

for item in tqdm(items):
    q_num += 1
    gold = np.array(item['SUP_EVIDENCE'])
    evidence_num += np.count_nonzero(gold == 1)

print("{} questions".format(q_num))
print("{} evidence sentences".format(evidence_num))
print("evidence/question= {}".format(evidence_num/q_num))

165 questions
236 evidence sentences
evidence/question= 1.4303030303030304


In [111]:
from fgc_support_retri.fgc_preprocess import keyword2se

with open(config.FGC_TRAIN) as f:
    items = ujson.load(f)
for item in tqdm(items):
    try:
        item = keyword2se(item)
    except:
        print(item['DID'])

D036
D040
aspan error:  2019 
{'QTYPE': '基础题', 'AMODE': 'Multi-Spans-Extraction', 'ASPAN': [{'text': '木雕博馆广场', 'end': 36, 'start': 30}, {'text': '水美商圈', 'end': 41, 'start': 37}, {'text': ' 2019', 'end': 4, 'start': -1}, {'text': '三义木雕艺术节', 'end': 11, 'start': 4}, {'text': '擧行', 'end': 45, 'start': 43}], 'ATYPE': 'Location', 'SHINT': [{'text': '2019三义木雕艺术节系列活动将从10月10日起一连四天，在木雕博馆广场及水美商圈盛大擧行，现场除了国内外的木雕作品展示，还有艺文表演、茶席体验等，民众不妨趁著双十假期到三义感受慢城之美', 'end': 93, 'start': 0}, {'text': ' 记者\u3000陈怡伶\u3000报导 2019三义木雕艺术节系列活动以「艺动-木艺慢活嘉年华」为主题，结合三义慢城意象，以及独特木雕与茶文化特色，邀请民众来一趟深度探索之旅，三义木雕博物馆也配合这次活动，自国庆日起连续四天免费开放参观', 'end': 203, 'start': 94}], 'QTEXT': '2019三义木雕艺术节活动于三义的哪两个地点举行?', 'QID': 'D068Q05', 'ANSWER': [{'ATOKEN': [{'text': '木雕博馆广场', 'start': 30}, {'text': '水美商圈', 'start': 37}], 'ATEXT': '木雕博馆广场及水美商圈'}]}


In [128]:
items[0].keys()

dict_keys(['DTEXT', 'QUESTIONS', 'DID', 'SENTS'])

In [124]:
items[0]['SENTS']

[('苏轼（1037年1月8日－1101年8月24日），眉州眉山（今四川省眉山市）人，北宋时著名的文学家、政治家、艺术家、医学家。', 0, 62),
 ('字子瞻，一字和仲，号东坡居士、铁冠道人。', 62, 82),
 ('嘉佑二年进士，累官至端明殿学士兼翰林学士，礼部尚书。', 82, 108),
 ('南宋理学方炽时，加赐谥号文忠，复追赠太师。', 108, 129),
 ('有《东坡先生大全集》及《东坡乐府》词集传世，宋人王宗稷收其作品，编有《苏文忠公全集》。', 129, 172),
 ('其散文、诗、词、赋均有成就，且善书法和绘画，是文学艺术史上的通才，也是公认韵文散文造诣皆比较杰出的大家。', 173, 225),
 ('苏轼的散文为唐宋四家（韩愈、柳宗元、欧苏）之末，与唐代的古文运动发起者韩愈并称为「韩潮苏海」，也与欧阳修并称「欧苏」；更与父亲苏洵、弟苏辙合称「三苏」，父子三人，同列唐宋八大家。',
  225,
  314),
 ('苏轼之诗与黄庭坚并称「苏黄」，又与陆游并称「苏陆」；其词「以诗入词」，首开词坛「豪放」一派，振作了晚唐、五代以来绮靡的西昆体余风。',
  314,
  379),
 ('后世与南宋辛弃疾并称「苏辛」，惟苏轼故作豪放，其实清朗；其赋亦颇有名气，最知名者为贬谪期间借题发挥写的前后《赤壁赋》。', 379, 438),
 ('宋代每逢科考常出现其文命题之考试，故当时学者曰：「苏文熟，吃羊肉、苏文生，嚼菜羹」。', 438, 480),
 ('艺术方面，书法名列「苏、黄、米、蔡」北宋四大书法家（宋四家）之首；其画则开创了湖州画派；并在题画文学史上占有举足轻重的地位。', 480, 542)]

In [127]:
len(items)

51

In [133]:
d_num = 0
q_num = 0
s_num = 0
evidence_num = 0

for i, item in enumerate(items):
    d_num += 1
    
    if 'SENTS' not in item.keys():
        print(item['DID'])
        continue
    s_num += len(item['SENTS'])
    
    for question in item['QUESTIONS']:
        q_num += 1
        gold = np.array(question['SUP_EVIDENCE'])
        evidence_num += np.count_nonzero(gold == 1)
        
print("{} documents".format(d_num))
print("{} sentences".format(s_num))
print("Sentence / document = {}".format(s_num/d_num))

print("{} questions".format(q_num))
print("{} evidence sentences".format(evidence_num))
print("evidence/question= {}".format(evidence_num/q_num))
    

D036
D040
51 documents
452 sentences
Sentence / document = 8.862745098039216
332 questions
491 evidence sentences
evidence/question= 1.4789156626506024
